In [1]:
pwd()

'c:\\Users\\hp\\Desktop\\CodeBase\\deep_CNN_Classifier\\research'

In [2]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\hp\\Desktop\\CodeBase\\deep_CNN_Classifier'

In [3]:

from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories
import tensorflow as tf

ModuleNotFoundError: No module named 'deepClassifier'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)-> EvaluationConfig:
        eval_config= EvaluationConfig(
            path_of_model= "artifacts/training/model.h5",
            training_data= "artifacts/data_ingestion/PetImages",
            params_image_size= self.params.IMAGE_SIZE,
            params_batch_size= self.params.BATCH_SIZE
        )
        return eval_config 

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path)-> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self.config.path_of_model):
       model= self.load_model("artifacts/training/model.h5")
       self._valid_generator()
       model.evaluate(self.valid_generator)

In [ ]:
try:
    config = ConfigurationManager()
    val_config= config.get_validation_config()
    evaluation= Evaluation(val_config)
    evaluation.evaluation()
    
except Exception as e:
    raise e